In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
if "../src" not in sys.path:
    sys.path.append("../../src")
    
import json
import pandas as pd
from pathlib import Path
from itertools import chain
from typing import cast, Dict
from tqdm.auto import tqdm
import numpy as np
from praatio import textgrid
from weave import Utterance, Speech, BuildUtt

## Data dependencies

*.mfa.json is generated by `etc/60.18`

In [4]:
!ls ../../data/aligned_xianzai/mfa_json/*.mfa.json | wc -l
!cd ../../data/aligned_xianzai/mfa_json && sha1sum *.mfa.json | sha1sum

852


20a3dd228b57ee027f3884ae0dcd26dc2aee79aa  -


In [5]:
from weave import MfaAgent
CORPUS_NAME = "xianzai"
mfa_dir = Path(f"~/Documents/MFA/{CORPUS_NAME}").expanduser()
agent = MfaAgent(str(mfa_dir), CORPUS_NAME, 
                 speech_json_dir="../../data/aligned_xianzai/mfa_json")

STDERR: gmm-copy --binary=false /home/seantyh/Documents/MFA/xianzai/alignment/final.mdl - 
LOG (gmm-copy[5.5.1068]:main():gmmbin/gmm-copy.cc:75) Written model to -



## Sandbox AM

In [6]:
agent.phones

{'<eps>': 0,
 'sil': 1,
 'sil_B': 2,
 'sil_E': 3,
 'sil_I': 4,
 'sil_S': 5,
 'spn': 6,
 'spn_B': 7,
 'spn_E': 8,
 'spn_I': 9,
 'spn_S': 10,
 'a_B': 11,
 'a_E': 12,
 'a_I': 13,
 'a_S': 14,
 'aj_B': 15,
 'aj_E': 16,
 'aj_I': 17,
 'aj_S': 18,
 'aj˥˥_B': 19,
 'aj˥˥_E': 20,
 'aj˥˥_I': 21,
 'aj˥˥_S': 22,
 'aj˥˩_B': 23,
 'aj˥˩_E': 24,
 'aj˥˩_I': 25,
 'aj˥˩_S': 26,
 'aj˦_B': 27,
 'aj˦_E': 28,
 'aj˦_I': 29,
 'aj˦_S': 30,
 'aj˧˥_B': 31,
 'aj˧˥_E': 32,
 'aj˧˥_I': 33,
 'aj˧˥_S': 34,
 'aj˨_B': 35,
 'aj˨_E': 36,
 'aj˨_I': 37,
 'aj˨_S': 38,
 'aj˨˩˦_B': 39,
 'aj˨˩˦_E': 40,
 'aj˨˩˦_I': 41,
 'aj˨˩˦_S': 42,
 'aj˩_B': 43,
 'aj˩_E': 44,
 'aj˩_I': 45,
 'aj˩_S': 46,
 'aw_B': 47,
 'aw_E': 48,
 'aw_I': 49,
 'aw_S': 50,
 'aw˥˥_B': 51,
 'aw˥˥_E': 52,
 'aw˥˥_I': 53,
 'aw˥˥_S': 54,
 'aw˥˩_B': 55,
 'aw˥˩_E': 56,
 'aw˥˩_I': 57,
 'aw˥˩_S': 58,
 'aw˦_B': 59,
 'aw˦_E': 60,
 'aw˦_I': 61,
 'aw˦_S': 62,
 'aw˧˥_B': 63,
 'aw˧˥_E': 64,
 'aw˧˥_I': 65,
 'aw˧˥_S': 66,
 'aw˨_B': 67,
 'aw˨_E': 68,
 'aw˨_I': 69,
 'aw˨_S': 70,
 'aw

In [15]:
import pandas as pd
triple_df = pd.DataFrame(agent.tmodel.triples)
ctab = pd.crosstab(triple_df[2], triple_df[1])
((ctab!=0).sum(axis=0)).sort_values()
ctab


1,0,1,2,3,4
2,,,,,
0,5,0,0,0,0
1,5,0,0,0,0
2,0,32,0,0,0
3,32,0,0,0,0
4,32,0,0,0,0
...,...,...,...,...,...
5971,0,0,4,0,0
5972,0,4,0,0,0
5973,0,4,0,0,0


In [9]:
agent.tmodel.am.npdf

5976

In [29]:
len([x for x in agent.phones.keys() if not x.startswith("#")])

543

In [20]:
len(agent.tmodel.state2tid)

91361

In [21]:
len(agent.tmodel.tid2state)

180773

In [17]:
agent.tmodel.topo.entries

[TopologyEntry(
   phone_ids=1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
   hmm_states=
     HmmState(state_idx=0,pdf_class=0  transitions=[(0, 0.25), (1, 0.25), (2, 0.25), (3, 0.25)]
     HmmState(state_idx=1,pdf_class=1  transitions=[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]
     HmmState(state_idx=2,pdf_class=2  transitions=[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]
     HmmState(state_idx=3,pdf_class=3  transitions=[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]
     HmmState(state_idx=4,pdf_class=4  transitions=[(4, 0.75), (5, 0.25)]
     HmmState(state_idx=5,pdf_class=None  transitions=[]),
 TopologyEntry(
   phone_ids=219, 220, 221, 222, 303, 304, 305, 306, 315, 316...,
   hmm_states=
     HmmState(state_idx=0,pdf_class=0  transitions=[(1, 0.3333333), (2, 0.3333333), (3, 0.3333333)]
     HmmState(state_idx=1,pdf_class=1  transitions=[(1, 0.5), (2, 0.5)]
     HmmState(state_idx=2,pdf_class=2  transitions=[(3, 1.0)]
     HmmState(state_idx=3,pdf_class=None  transitions=[]),
 TopologyEntry(
  